## Advent of code 2023 day 1-10
See https://adventofcode.com/

In [ ]:
# note that this notebook requires the .venv environment (which is set up with pypy3.10-v7.3.13-win64)
# to activate it from a git bash shell: source .venv/Scripts/activate

import collections
import itertools
import functools
import re
import copy
import math
import sys
import time
import json
import heapq
import bisect
import random
import sortedcontainers

import zio

In [ ]:
# version check and timestamp
# NB the timestamp supports ranking using an honor system, before starting include this line
# in the header of your solution (which should start with a line like # 2019 day 2), then whenever you want save
# a private leaderboard json file, and run python privaterank.py filename.json

print(f'python version: {sys.version}')
print(f'# start_ts={int(time.time())}')

In [ ]:
# 2023 day 6 part 1
# mv ~/Downloads/input* data_src/2023-day-6-input.txt
# big input file looks like: only a few races, small numbers
# idea: part 1 parse as time,distance pairs, then iterate over hold-down times and just count ways to win

sample2='''
Time:      7  15   30
Distance:  9  40  200
'''

sample1=open('data_src/2023-day-6-input.txt').read()
lines=[s.split() for s in sample1.splitlines() if len(s)>0 ]
assert len(lines)==2
data=[]
for i in range(1, len(lines[0])):
    data.append( (int(lines[0][i]), int(lines[1][i]))) # time, distance

# part 1
score=1
for race in data:
    count=0
    t, dist=race
    for t1 in range(t+1):
        t2=t-t1
        dt=t1*t2
        if dt>dist:
            count+=1
    score*=count
print(f'part 1: {score=}')

In [ ]:
# 2023 day 6 part 2
# idea: parse without kerning/spacing as a single pair, then count winning ways as before
# (even this very inefficient way is fast enough on the given data, taking only 0.4s)

sample1=open('data_src/2023-day-6-input.txt').read()
lines=[s.replace(' ', '') for s in sample1.splitlines() if len(s)>0 ]
assert len(lines)==2
t=int(lines[0].split(':')[1])
dist=int(lines[1].split(':')[1])
print(f'part 2: {t=}, {dist=}')

count=0
for t1 in range(t+1):
    t2=t-t1
    dt=t1*t2
    if dt>dist:
        count+=1
print(f'part 2: {count=}')

In [ ]:
# 2023 day 5 part 1
# mv ~/Downloads/input* data_src/2023-day-5-input.txt
# big input file looks like: sections with big numbers
# idea: part 1 parse as sections of tuples, then put the seeds through the mappings one by one

sample2='''
seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4
'''

def apply_map(item, ranges):
    '''map a single item value through the specified map (as list of lists of 3 elements),
    returning the mapped value'''
    #print(f'apply_map, {item=}, {ranges=}')
    for r in ranges:
        dest0, src0, sz=r
        if src0 <= item < src0+sz:
            return dest0+item-src0
    return item

def seed_to_loc(seed, data):
    '''map a single seed value through all the mappings in data, returning its location'''
    #mapped=[seed]
    item=seed
    for key in list(data.keys())[1:]:
        item=apply_map(item, data[key])
        #mapped.append(item)
    #print(f'{mapped=}')
    return item

sample1=open('data_src/2023-day-5-input.txt').read()
groups=zio.get_line_groups(sample1.splitlines(), nostrip=False)
data={} # maps map name to list of lists except first is just a list, assumes ordered dict
for gr in groups:
    if len(gr)==1:
        tup=gr[0].split(':')
        name=tup[0]
        row=tup[1].split()
        data[name]=[int (n) for n in row]
    else:
        name=gr[0][:-1]
        rows=[ [int(n) for n in s.split()] for s in gr[1:] ]
        data[name]=rows

# part 1
seeds=data['seeds']
min_loc=None
for seed in seeds:
    loc=seed_to_loc(seed, data)
    if min_loc is None or loc<min_loc:
        min_loc=loc
print(f'part 1: {min_loc=}')

In [ ]:
# 2023 day 5 part 2
# idea: put the seeds through the mappings in ranges, splitting up the ranges as needed

def apply_map2(iv, ranges):
    '''map a single interval (pair of minval,maxval) through the specified map (as list of lists of 3 elements),
    returning the mapped value as a list of intervals (each a pair of minval,maxval)'''
    ivmin,ivmax=iv
    assert ivmax>=ivmin
    res=[]
    for r in ranges: # for each range element split interval into a mapped part and a non-mapped remainder
        destmin, srcmin, rsz=r
        srcmax=srcmin+rsz-1
        assert srcmax>=srcmin
        if ivmin>=srcmin and ivmax<=srcmax: # matched pair range is ivmin to ivmax
            res.append( (destmin+ivmin-srcmin, destmin+ivmax-srcmin) )
            ivmin=None # nothing remaining
            break
        elif srcmin <= ivmin <= srcmax: # matched pair range is ivmin to srcmax
            res.append( (destmin+ivmin-srcmin, destmin+srcmax-srcmin) )
            ivmin=srcmax+1 # remaining srcmax+1 to ivmax
        elif srcmin <= ivmax <= srcmax: # matched pair range is srcmin to ivmax
            res.append( (destmin+srcmin-srcmin, destmin+ivmax-srcmin) )
            ivmax=srcmin-1 # remaining ivmin to srcmin-1
    if ivmin is not None:
        res.append( (ivmin, ivmax) )
    return res

def seed_to_loc2(seed0, sz, data):
    '''map a single seed range (start seed and size) through all the mappings in data,
    returning its minimum location'''
    ivs=[(seed0, seed0+sz-1)]
    #print(f'seed_to_loc2 start: {ivs=}')
    for key in list(data.keys())[1:]:
        res=[]
        for iv in ivs:
            res_ivs=apply_map2(iv, data[key])
            res.extend(res_ivs)
        ivs=res
        #print(f'seed_to_loc2 after {key=}: {ivs=}')
    res=min([iv[0] for iv in res])
    #print(f'seed_to_loc2 res: {res=}')
    return res

# part 2
seeds=data['seeds']
assert (len(seeds) % 2) == 0
min_loc=None
for i in range(0, len(seeds), 2):
    seed0=seeds[i]
    sz=seeds[i+1]
    loc=seed_to_loc2(seed0, sz, data)
    if min_loc is None or loc<min_loc:
        min_loc=loc
print(f'part 2: {min_loc=}')

In [ ]:
# 2023 day 5 part 2 naive
# definitely takes too long :-)

# part 2
seeds=data['seeds']
assert (len(seeds) % 2) == 0
min_loc=None
for i in range(0, len(seeds), 2):
    seed0=seeds[i]
    sz=seeds[i+1]
    for seed in range(seed0, seed0+sz):
        loc=seed_to_loc(seed, data)
        if min_loc is None or loc<min_loc:
            min_loc=loc
print(f'part 2 naive: {min_loc=}')

In [ ]:
# 2023 day 4
# mv ~/Downloads/input* data_src/2023-day-4-input.txt
# big input file looks like: 202 cards
# idea: part 1 parse into dicts with sets, then use intersection and pow
#  part 2: keep a Counter for the cards

sample2='''
Card   1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53
Card  2: 13 32 20 16 61 | 61 30 68 82 17 32 24 19
Card 3:  1 21 53 59 44 | 69 82 63 72 16 21 14  1
Card 4: 41 92 73 84 69 | 59 84 76 51 58  5 54 83
Card 5: 87 83 26 28 32 | 88 30 70 12 93 22 82 36
Card 6: 31 18 13 56 72 | 74 77 10 23 35 67 36 11
'''

sample1=open('data_src/2023-day-4-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
cards = {} # maps id to { 'win': set(), 'own': set() }
for s in lines:
    tup = s.split(':')
    id = int(tup[0].split()[1])
    tup = tup[1].split('|')
    win = { int(n) for n in tup[0].split() }
    own = { int(n) for n in tup[1].split() }
    cards[id]= dict(id=id, win=win, own=own )

# part 1
score=0
for card in cards.values():
    own=card['own']
    win=card['win']
    overlap=own.intersection(win)
    if len(overlap)>0:
        score += pow(2, len(overlap)-1)
print(f'part 1: {score=}')

# part 2
count=collections.Counter() # maps card id to count of cards won
for card in cards.values():
    count[card['id']]=1
idlist=sorted(count.keys())
for i,id in enumerate(idlist):
    card=cards[id]
    own=card['own']
    win=card['win']
    overlap=own.intersection(win)
    for j in range(i+1, min(i+len(overlap)+1, len(idlist))):
        id2=idlist[j]
        count[id2] += count[id]
score=sum(count.values())
print(f'part 2: {score=}')

In [ ]:
# 2023 day 3
# mv ~/Downloads/input* data_src/2023-day-3-input.txt
# big input file looks like: a map?
# idea: part 1 parse as basic map, then char by char left to right build up numbers while
#  scanning for symbols around the number chars and recording them
# part 2: invert the list of numbers to a map of symbols, then iterate over that

sample2='''
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
'''

def gen_numbers(lines):
    '''parse out numbers with their symbols, return list of lists, first is
    number, followed by a set of symbols with their coords'''
    res=[]
    curnum=''
    cursyms=set()
    for y0, row in enumerate(lines):
        for x0,c in enumerate(row):
            if '0' <= c <= '9':
                curnum+=c
                for y in range(y0-1, y0+2):
                    for x in range(x0-1, x0+2):
                        if y>=0 and y<len(lines) and x>=0 and x<len(lines[y]):
                            potsym=lines[y][x]
                            if not (potsym=='.' or ('0' <= potsym <= '9')):
                                potsymtup=(potsym, x, y)
                                cursyms.add(potsymtup)
            else:
                if len(curnum)>0:
                    res.append( [int(curnum), cursyms] )
                    curnum=''
                    cursyms=set()
        if len(curnum)>0:
            res.append( [int(curnum), cursyms] )
            curnum=''
            cursyms=set()
    return res

sample1=open('data_src/2023-day-3-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]

# part 1
numlist=gen_numbers(lines)
totalparts=sum([ tup[0] for tup in numlist if len(tup[1]) > 0 ])
print(f'part 1: {totalparts=}')

# part 2
starmap={} # maps startup (sym, x, y) to set of surrounding nums
           #  (part numbers, which are in this context unique per symbol)
for tup in numlist:
    num=tup[0]
    for symtup in tup[1]:
        if symtup[0] != '*':
            continue
        ls=starmap.setdefault(symtup, set())
        ls.add(num)
starsum=0
for k,v in starmap.items():
    if len(v)==2:
        v=list(v)
        starsum+=v[0]*v[1]
print(f'part 2: {starsum=}')

In [ ]:
# 2023 day 2
# mv ~/Downloads/input* data_src/2023-day-2-input.txt
# big input file looks like: 100 games with color counts
# idea: part 1 parse by descending splits (runs in games, elements in runs),
#  then check the run limits along the way
# part 2: again iterate, now calculating the minimum (actually maximum) per game

sample2='''
Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green
Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue
Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red
Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red
Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green
'''

sample1=open('data_src/2023-day-2-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]

# part 1
data=[]
okgameids=set()
maxgam1={'red': 12, 'green': 13, 'blue': 14}
for s in lines:
    game={}
    tup=s.split(':')
    game['id']=int(tup[0][5:])
    okgameids.add(game['id'])
    tup=tup[1].split(';')
    runs=[]
    for rn in tup:
        rund={}
        elems=rn.split(',')
        for elem in elems:
            pair=elem.split()
            color=pair[1]
            cnt=int(pair[0])
            rund[color]=cnt
            if color in maxgam1:
                maxcnt=maxgam1[color]
                if cnt>maxcnt:
                    okgameids.discard(game['id'])
        runs.append(rund)
    game['runs']=runs
    data.append(game)
print(f'part 1: {sum(okgameids)}')

# part 2
powsum=0
for game in data:
    mincnt={}
    for run in game['runs']:
        for k,v in run.items():
            assert k in {'red', 'green', 'blue'}
            if k in mincnt:
                mincnt[k]=max(v, mincnt[k])
            else:
                mincnt[k]=v
    pow=1
    for v in mincnt.values():
        pow*=v
    powsum+=pow
print(f'part 2: {powsum=}')

In [ ]:
# 2023 day 1
# mv ~/Downloads/input* data_src/2023-day-1-input.txt
# big input file looks like: all kinds of words with digits in them
# idea: part 1 parse as lines, then iterate over s and reversed(s) to find digits
# part 2: map of digit words, then for each position check digit and digit words

sample2='''
1abc2
pqr3stu8vwx
a1b2c3d4e5f
treb7uchet
'''

sample3='''
two1nine
eightwothree
abcone2threexyz
xtwone3four
4nineeightseven2
zoneight234
7pqrstsixteen
'''

sample1=open('data_src/2023-day-1-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]

# part 1
n=0
for s in lines:
    digits=''
    for c in s:
        if '0' <= c <= '9':
            digits+=c
            break
    for c in reversed(s):
        if '0' <= c <= '9':
            digits+=c
            break
    if len(digits)!=2:
        continue
    n+=int(digits)
print(f'part 1: {n=}')

# part 2
digmap={
    'one': 1,
    'two': 2,
    'three': 3,
    'four': 4,
    'five': 5,
    'six': 6,
    'seven': 7,
    'eight': 8,
    'nine': 9,
}
n=0
for s in lines:
    digits=''
    for i in range(len(s)):
        c=s[i]
        if '0' <= c <= '9':
            digits+=c
        else:
            for k, v in digmap.items():
                if s[i:].startswith(k):
                    digits+=str(v)
                    break    
    n+=int(digits[0]+digits[-1])
print(f'part 2: {n=}')

In [ ]:
# TEMPLATE
# 2023 day 1
# start_ts=RUN FIRST CELL TO GET TIME CODE BEFORE OPENING THE ASSIGNMENT
# mv ~/Downloads/input* data_src/2023-day-1-input.txt
# big input file looks like: 
# idea: part 1 parse ..., then ...

sample2='''

'''

sample1=open('data_src/2023-day-1-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
groups=zio.get_line_groups(sample1.splitlines(), nostrip=False)
data=[ int(s) for s in lines[0].split(',') ]
data=[ s.split() for s in lines ]
data=[ [cmd, int(num), 0] for cmd, num in data ]
data=[ result.group(1, 2, 3, 4, 5, 6, 7) for s in lines if (result:= re.match(r'(\w+)\s*x=([\d\-]+)\.\.([\d\-]+),y=([\d\-]+)\.\.([\d\-]+),z=([\d\-]+)\.\.([\d\-]+)', s)) ]
data=[ (row[0], int(row[1]), int(row[2]), int(row[3]), int(row[4]), int(row[5]), int(row[6]) ) for row in data ]
# template, remove what's not needed